# NBA Statistics

Documents the use of LangChain Agents/Tools to help aid in answering questions based on recent events.

In [32]:
!pip install langchain openai python-dotenv bs4 requests

In [33]:
from dotenv import load_dotenv
load_dotenv()

True

In [36]:
from langchain.llms import OpenAI

llm = OpenAI(
    temperature=0,
    verbose=True,
    model_name="text-davinci-003"
)

## Without a source

The LLM was trained upto a given point in time, it is not aware of recent events.

In [37]:
question = "Was the 2023 NBA season MVP part of the team that won the NBA championship that year?"

In [38]:
llm(question)

'\n\nIt is too early to tell.'

It can also be completely wrong...

In [39]:
llm("Was the 1998 NBA season MVP part of the team that won the NBA championship that year?")

'\n\nNo, the 1998 NBA season MVP was Michael Jordan of the Chicago Bulls, but the NBA championship that year was won by the San Antonio Spurs.'

In [40]:
llm("Who won the 1998 NBA season MVP, what team where they on?, what team won the NBA championship that year?")

'\n\nThe 1998 NBA season MVP was Michael Jordan of the Chicago Bulls. The Chicago Bulls also won the NBA championship that year.'

## Adding a source

We can improve the models answer by providing tools for it to use to lookup recent events within a given domain.

> "You Don’t Have to Know Everything. You Just Have to Know Where to Find It."
>
> -- <cite>Albert Einstein</cite>

In [48]:
from langchain.agents import AgentType, load_tools, initialize_agent, Tool
from bs4 import BeautifulSoup
import requests

def search_statmuse(query: str) -> str:
    response = requests.get(f'https://www.statmuse.com/nba/ask/{query}')
    soup = BeautifulSoup(response.content, "html.parser")
    return soup.find("h1", class_="nlg-answer").text

statmuse_tool = Tool(
    name = "Statmuse",
    func = search_statmuse,
    description = """
        NBA basketball search engine. Use this if the question is related to NBA basktetball stats and trivia.
        For example: 'who is the highest scoring player in the 2023 season?', 
        You must:
        - Always specify a year or timeframe with your search. 
        - Only search for a single player, MVP or team in a single search.
    """
)

tools = load_tools(["llm-math"], llm=llm) + [statmuse_tool]
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [49]:
agent.run(question)



> Entering new AgentExecutor chain...
 I need to find out who the MVP was and which team they were on
Action: Statmuse
Action Input: Who was the 2023 NBA season MVP?
Observation: 
Joel Embiid won the MVP award in 2022-23.

Thought: I need to find out which team Joel Embiid was on
Action: Statmuse
Action Input: What team was Joel Embiid on in the 2023 season?
Observation: 
Joel Embiid played for the 76ers in 2022-23.

Thought: I need to find out which team won the NBA championship in 2023
Action: Statmuse
Action Input: Who won the NBA championship in 2023?
Observation: 
The Denver Nuggets won in the 2023 NBA Finals against the Miami Heat, 4 games to 1.

Thought: I now know the final answer
Final Answer: No, Joel Embiid was not part of the team that won the NBA championship in 2023.

> Finished chain.


'No, Joel Embiid was not part of the team that won the NBA championship in 2023.'

In [50]:
agent.run("Was the 1998 NBA season MVP part of the team that won the NBA championship that year?")



> Entering new AgentExecutor chain...
 I need to find out who the MVP was and which team won the championship
Action: Statmuse
Action Input: Who was the 1998 NBA season MVP?
Observation: 
Michael Jordan won the MVP award in 1997-98.

Thought: I need to find out which team won the championship that year
Action: Statmuse
Action Input: Who won the 1998 NBA championship?
Observation: 
The Chicago Bulls won in the 1998 NBA Finals against the Utah Jazz, 4 games to 2.

Thought: I now know the final answer
Final Answer: Yes, Michael Jordan was part of the team that won the 1998 NBA championship.

> Finished chain.


'Yes, Michael Jordan was part of the team that won the 1998 NBA championship.'

## Combining tools

We now can combine the abilities to; reason about a question, find the relevant information, and then compute a given answer.

In [52]:
agent.run("How many combined points did the 2023 NBA season MVP, and the 2023 NBA finals MVP score for the entire 2023 season?")



> Entering new AgentExecutor chain...
 I need to find out who the MVPs were and then add up their points
Action: Statmuse
Action Input: Who was the 2023 NBA season MVP?
Observation: 
Joel Embiid won the MVP award in 2022-23.

Thought: Now I need to find out who the Finals MVP was
Action: Statmuse
Action Input: Who was the 2023 NBA Finals MVP?
Observation: 
Nikola Jokic won the Finals MVP award once in the 2023 NBA Finals.

Thought: Now I need to find out how many points each of them scored in the 2023 season
Action: Statmuse
Action Input: How many points did Joel Embiid score in the 2023 season?
Observation: 
Joel Embiid scored 2,183 points in 2022-23.

Thought: Now I need to find out how many points Nikola Jokic scored in the 2023 season
Action: Statmuse
Action Input: How many points did Nikola Jokic score in the 2023 season?
Observation: 
Nikola Jokic dropped 1,690 points in 2022-23.

Thought: I now know the final answer
Final Answer: The combined points of the 2023 NBA season MVP 

'The combined points of the 2023 NBA season MVP and the 2023 NBA Finals MVP for the entire 2023 season was 3,873 points.'

In [53]:
tools = load_tools(["wikipedia"], llm=llm) + [statmuse_tool]
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent.run("Where was the NBA finals MVP for the 2023 season born, and how many total points did he score for the entire season?")



> Entering new AgentExecutor chain...
 I need to find out who the MVP was and then look up their birthplace and total points scored.
Action: Statmuse
Action Input: Who was the 2023 NBA Finals MVP?
Observation: 
Nikola Jokic won the Finals MVP award once in the 2023 NBA Finals.

Thought: I need to find out where Nikola Jokic was born and how many points he scored in the 2023 season.
Action: Wikipedia
Action Input: Nikola Jokic
Observation: Page: Nikola Jokić
Summary: Nikola Jokić (Serbian Cyrillic: Никола Јокић, pronounced [nǐkola jôkitɕ] (listen); born February 19, 1995) is a Serbian professional basketball player, who is a center for the Denver Nuggets of the National Basketball Association (NBA). Nicknamed "The Joker," and hailed as one of the biggest draft steals in NBA history, he is regarded as one of the greatest players and centers of all time. A five-time NBA All-Star, he has been named to the All-NBA Team on five occasions (including three first-team selections), and won the

'Nikola Jokić was born in Sombor, Serbia on February 19, 1995 and he scored a total of 1,845 points in the 2023 season.'